In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from spn.algorithms.Inference import log_likelihood
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian, Bernoulli
import numpy as np

newnames = ['telephoning',
 'medicine',
 'money',
 'traveling',
 'outside',
 'grocery',
 'cooking',
 'laundry',
 'light',
 'heavy',
 'toilet',
 'bathing',
 'dressing',
 'inside',
 'bed',
 'eating']

frame = pd.read_csv("nltcs/2to16disabilityNLTCS.txt", sep='\t').drop(['PERCENT'], axis=1)
frame = frame.rename({old: new  for old, new in zip(frame.columns, newnames)}, axis=1)
print ("%d rows" % len(frame))

D = frame.to_numpy()
expanded = []
for row in D:
    expanded.append([list(row[0:-1])] * row[-1])
D = np.concatenate(expanded, axis=0)
frame = pd.DataFrame(D, columns = frame.columns[:-1])
print ("%d rows" % len(frame))
train, test = train_test_split(frame, random_state = 0)
print (len(frame), len(train), len(test))

3152 rows
21574 rows
21574 16180 5394


In [4]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules


frequent_itemsets = fpgrowth(train, use_colnames=True, min_support = 0.25)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(heavy),(outside),0.675402,0.552596,0.473177,0.700586,1.267809,0.099953,1.494264
1,(outside),(heavy),0.552596,0.675402,0.473177,0.856280,1.267809,0.099953,2.258547
2,(traveling),(outside),0.493634,0.552596,0.386341,0.782647,1.416310,0.113561,2.058420
3,(traveling),(heavy),0.493634,0.675402,0.436156,0.883561,1.308200,0.102754,2.787706
4,"(traveling, heavy)",(outside),0.436156,0.552596,0.356613,0.817628,1.479613,0.115595,2.453249
...,...,...,...,...,...,...,...,...,...
160,(laundry),"(traveling, grocery, heavy)",0.353585,0.364400,0.257355,0.727845,1.997376,0.128508,2.335430
161,"(laundry, bathing)",(heavy),0.261743,0.675402,0.259394,0.991027,1.467315,0.082613,36.175623
162,"(laundry, heavy)",(bathing),0.345179,0.440791,0.259394,0.751477,1.704837,0.107242,2.250131
163,(laundry),"(bathing, heavy)",0.353585,0.394129,0.259394,0.733613,1.861355,0.120036,2.274403


In [5]:
confidences = []
for i, row in rules.iterrows():
    acondition = None
    for name in row.antecedents:
        if acondition is None:
            acondition = (test[name] == 1)
        else:
            acondition = acondition & (test[name] == 1)
    
    ccondition = None
    for name in row.consequents:
        if ccondition is None:
            ccondition = (test[name] == 1)
        else:
            ccondition = ccondition & (test[name] == 1)
    
    confidences.append (len(test[acondition & ccondition]) / len(test[acondition]))

print ("test condidence = %.2f, alen = %.2f, clen = %.2f" % (np.mean(confidences), 
                                                            rules.antecedents.apply(len).mean(), 
                                                            rules.consequents.apply(len).mean()))

test condidence = 0.83, alen = 1.96, clen = 1.28
